# Benchmark for bayesian optimization

Warning: By default Botorch maximizes the function. I will need to take the negative of my gt_func

In [1]:
from test_functions import Hart6
from bayes_lib import ExactGPModel, train_hyper_params
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.constraints import Interval
import torch
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from scipy.stats import truncnorm
def get_test_x(n, mu=0.5, sigma=0.25, lower=0.0, upper=1.0):
    """ returns a n x d tensor containing random points taking from truncated 
    normal distributions of means mu, and standard deviation sigma (mu and sigma are floats or lists of length d).
    The distribution is truncated between lower and upper bounds.
    for more information on the truncated normal distribution see: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.truncnorm.html
Parameters
==============
n: int
    number of points to return
mu: float or array-like, default: 0.5
    list of means of length d
sigma: float or array-like, default: 0.25
    list of standard deviations (must be the same length as mu)
lower: float, default: 0.0
    lower bound
uper: float, default: 1.0

Returns
==============
tensor of shape (n, d)

    """
    if type(mu) == int or type(mu) == float:
        mu = [mu]
    if type(sigma) == int or type(sigma) == float:
        sigma = [sigma]
    if len(mu) != len(sigma):
        raise ValueError(f"mu and sigma must have the same length, but got len(mu)={len(mu)}, len(sigma)={len(sigma)}")
        
    ndim = len(mu)
    out = []
    for i in range(ndim):
        X = truncnorm(
            (lower - mu[i]) / sigma[i], (upper - mu[i]) / sigma[i], loc=mu[i], scale=sigma[i])
        out.append(torch.from_numpy(X.rvs(n)))

    return torch.stack(out).T.reshape(-1,1,ndim).float()

In [3]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.models.gpytorch import GPyTorchModel
from IPython.display import clear_output
class ExactGPModel(GPyTorchModel, gpytorch.models.ExactGP):
    _num_outputs = 1
    
    def __init__(self, train_x, train_y, likelihood):
        # super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        # self.to(train_x)
        
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
   

In [12]:
# Bayes opt loop
Model = ExactGPModel
gt_func = Hart6(return_negative=True) # objective function
likelihood = GaussianLikelihood(noise_constraint=Interval(0.0,1e-14))
n = int(1e6) # number of random evaluation points of the test function


In [14]:
train_x = torch.rand(6).reshape(-1,6)
train_y = gt_func.f(train_x)
max_iter = 200
it = 0
best_f = -1e5
rel_tol = 1e-06
test_x = get_test_x(n, mu=[0.5 for i in range(6)],  sigma=[0.25 for i in range(6)])
error_gaps = np.zeros(max_iter)
for i in range(max_iter):
    # Run the forward model with hyperparam opt
    model = Model(train_x, train_y, likelihood)
    train_hyper_params(model, likelihood)

    # Run the aquisition method
    EI = ExpectedImprovement(model, best_f=best_f, maximize=True)
    for j in range(2):
        # To limit the number of evaluation of ei, I evaluate it over several pass,
        # where each pass uses samples taken from a normal distribution
        # but everytime the mean is updated as the max of ei, and the standard deviation is decreased
        if j == 0:
            test_x = get_test_x(n, mu=[0.5 for i in range(6)],  sigma=[0.25 for i in range(6)])
        else:
            test_x = get_test_x(n, mu=x_new.reshape(-1),  sigma=[0.25/(2**j) for i in range(6)])
        ei = EI(test_x)
        x_new = test_x[ei.argmax().item()]
        # print(f"{j}, {x_new}, {ei[ei.argmax().item()]}")
        
    
#     # Add the suggested point to the training points

    y_new = gt_func.f(x_new)
    best_f = max(y_new.item(), best_f)

    train_x = torch.cat((train_x.reshape(-1,1), x_new.reshape(-1,1))).reshape(-1,6)
    train_y = torch.cat((train_y.reshape(-1,1), y_new.reshape(-1,1))).reshape(-1)
    
    print(f"{it+1}/{max_iter}: {best_f}, {train_y[-1].item()}, {np.log10(gt_func.error_gap(best_f))}", end="\r")
    error_gaps[it] = gt_func.error_gap(best_f)
    it += 1
plt.plot(error_gaps)
ax.set_yscale('log10')

/Users/abauville/opt/anaconda3/lib/python3.9/site-packages/gpytorch/distributions/multivariate_normal.py:259: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
ei

In [ ]:
train_y.shape

In [ ]:
model._train_targets[0]